In [62]:
import pandas as pd
import re

In [63]:
df_camera_video = pd.read_csv('camera_video.csv')
print(df_camera_video.head())

   product_id                                       camera_video
0       59254  4K@24 fps4K@25 fps4K@30 fps4K@60 fps1080p@25fp...
1       76439                                                NaN
2       59294  8K@24/30fps, 4K@30/60/120fps, 1080p@30/60/240f...
3       59257  Quay video 4K Dolby Vision ở tốc độ 24 fpsQuay...
4       59258  4K@24 fps, 4K@25 fps, 4K@30 fps, 4K@60 fps, 4K...


In [64]:
# Hàm để tách độ phân giải video và tốc độ khung hình riêng biệt
def extract_video_resolution_fps(video_info):
    if pd.isna(video_info):
        return pd.Series([None, None, None, None, None, None])
    
    # Tìm độ phân giải như 4K, 1080p, 720p, v.v., và tốc độ khung hình
    resolution_pattern = r'(\d{3,4}p|\d{1,2}K)'
    fps_pattern = r'(\d{2,3}\s?fps)'

    # Trích xuất độ phân giải và fps
    resolutions = re.findall(resolution_pattern, video_info)
    fps_values = re.findall(fps_pattern, video_info)
    
    # Chuẩn bị các giá trị đã trích xuất (độ phân giải 1, fps 1, độ phân giải 2, ...)
    resolution_fps_data = []
    
    for i in range(0, max(len(resolutions), len(fps_values)), 2):
        res = resolutions[i] if i < len(resolutions) else None
        fps = fps_values[i] if i < len(fps_values) else None
        resolution_fps_data.append(res)
        resolution_fps_data.append(fps)
    
    # Trả về series gồm 6 giá trị (độ phân giải video 1, fps video 1, độ phân giải video 2, ...)
    return pd.Series(resolution_fps_data[:6])

# Giả sử df là dataframe gốc của bạn
# Áp dụng hàm để tách dữ liệu thành nhiều cột
df_camera_video[['video_resolution_1', 'video_fps_1', 'video_resolution_2', 'video_fps_2', 'video_resolution_3', 'video_fps_3']] = df_camera_video['camera_video'].apply(extract_video_resolution_fps)

# Hiển thị dataframe đã cập nhật
df_camera_video.head()


,product_id,camera_video,video_resolution_1,video_fps_1,video_resolution_2,video_fps_2,video_resolution_3,video_fps_3
0,59254,4K@24 fps4K@25 fps4K@30 fps4K@60 fps1080p@25fp...,4K,24 fps,4K,30 fps,1080p,25fps
1,76439,NaN,None,None,None,None,None,None
2,59294,"8K@24/30fps, 4K@30/60/120fps, 1080p@30/60/240f...",8K,30fps,1080p,240fps,NaN,NaN
3,59257,Quay video 4K Dolby Vision ở tốc độ 24 fpsQuay...,4K,24 fps,720p,None,NaN,NaN
4,59258,"4K@24 fps, 4K@25 fps, 4K@30 fps, 4K@60 fps, 4K...",4K,24 fps,4K,30 fps,4K,100 fps


In [65]:
# Bản đồ độ phân giải phổ biến với tổng số điểm ảnh
resolution_to_pixels = {
    '4K': 3840 * 2160,
    '1080p': 1920 * 1080,
    '720p': 1280 * 720,
    '8K': 7680 * 4320,
    'HD': 1280 * 720,
    'FullHD': 1920 * 1080
}

# Hàm chuyển đổi độ phân giải thành số điểm ảnh
def resolution_to_pixel_count(resolution):
    if resolution in resolution_to_pixels:
        return resolution_to_pixels[resolution]
    return None

# Áp dụng hàm cho các cột video_resolution
df_camera_video['video_resolution_1'] = df_camera_video['video_resolution_1'].apply(resolution_to_pixel_count)
df_camera_video['video_resolution_2'] = df_camera_video['video_resolution_2'].apply(resolution_to_pixel_count)
df_camera_video['video_resolution_3'] = df_camera_video['video_resolution_3'].apply(resolution_to_pixel_count)

# Hiển thị DataFrame đã cập nhật
df_camera_video.head()

,product_id,camera_video,video_resolution_1,video_fps_1,video_resolution_2,video_fps_2,video_resolution_3,video_fps_3
0,59254,4K@24 fps4K@25 fps4K@30 fps4K@60 fps1080p@25fp...,8294400.0,24 fps,8294400.0,30 fps,2073600.0,25fps
1,76439,NaN,NaN,None,NaN,None,NaN,None
2,59294,"8K@24/30fps, 4K@30/60/120fps, 1080p@30/60/240f...",33177600.0,30fps,2073600.0,240fps,NaN,NaN
3,59257,Quay video 4K Dolby Vision ở tốc độ 24 fpsQuay...,8294400.0,24 fps,921600.0,None,NaN,NaN
4,59258,"4K@24 fps, 4K@25 fps, 4K@30 fps, 4K@60 fps, 4K...",8294400.0,24 fps,8294400.0,30 fps,8294400.0,100 fps


In [66]:
# resolution_1

# video_fps_1
df_camera_video['video_fps_1'] = df_camera_video['video_fps_1'].str.replace('fps', '', regex=False)

# resolution_2

# video_fps_2
df_camera_video['video_fps_2'] = df_camera_video['video_fps_2'].str.replace('fps', '', regex=False)

# resolution_3

# video_fps_3
df_camera_video['video_fps_3'] = df_camera_video['video_fps_3'].str.replace('fps', '', regex=False)

In [67]:
df_camera_video.drop(columns=['camera_video'], inplace=True)

In [68]:
# Đổi tên các cột
df_camera_video.rename(columns={
    'video_fps_1': 'video_fps_1 (fps)',
    'video_fps_2': 'video_fps_1 (fps)',
    'video_fps_3': 'video_fps_1 (fps)',
    'video_resolution_1': 'video_resolution_1 (pixels)',
    'video_resolution_2': 'video_resolution_2 (pixels)',
    'video_resolution_3': 'video_resolution_3 (pixels)'
}, inplace=True)

In [69]:
df_camera_video.fillna('N/A', inplace=True)
df_camera_video.head()

C:\Users\sangt\AppData\Local\Temp\ipykernel_5964\3997085783.py:1: FutureWarning: Setting an item of incompatible dtype is deprecated and will raise an error in a future version of pandas. Value 'N/A' has dtype incompatible with float64, please explicitly cast to a compatible dtype first.
  df_camera_video.fillna('N/A', inplace=True)


,product_id,video_resolution_1 (pixels),video_fps_1 (fps),video_resolution_2 (pixels),video_fps_1 (fps),video_resolution_3 (pixels),video_fps_1 (fps)
0,59254,8294400.0,24,8294400.0,30,2073600.0,25
1,76439,N/A,N/A,N/A,N/A,N/A,N/A
2,59294,33177600.0,30,2073600.0,240,N/A,N/A
3,59257,8294400.0,24,921600.0,N/A,N/A,N/A
4,59258,8294400.0,24,8294400.0,30,8294400.0,100


In [70]:
# Save the modified DataFrame to a new CSV file
df_camera_video.to_csv('camera_video_processed.csv', index=False, encoding='utf-8-sig')